In [4]:
!pip -q install duckdb pandas

import duckdb
import pandas as pd
from datetime import date, timedelta

con = duckdb.connect(database=":memory:")

# -------------------------------
# Create demo tables:
# silver_events (events-level)
# gold_products (daily & category metrics)
# -------------------------------

# Demo: gold_products (acts like product/day/category level metrics)
start = date(2024, 1, 1)
dates = [start + timedelta(days=i) for i in range(14)]

gold_products = pd.DataFrame({
    "event_date": dates * 3,
    "category_code": (["electronics"] * 14) + (["fashion"] * 14) + (["grocery"] * 14),
    "revenue": ([1200,1300,900,1500,1600,1400,1700,1800,1750,1600,1550,1900,2000,2100] +
                [600,650,620,700,720,680,740,770,760,800,820,830,850,870] +
                [300,320,310,330,340,350,360,370,380,390,395,400,410,420]),
    "views": ([1000,1100,950,1200,1250,1180,1300,1400,1350,1280,1260,1500,1600,1700] +
              [900,950,930,980,1000,990,1050,1080,1070,1100,1120,1130,1150,1200] +
              [800,820,810,830,840,860,880,890,900,910,920,930,940,950]),
    "purchases": ([40,42,35,45,50,48,55,60,58,54,52,65,70,72] +
                  [18,20,19,22,23,21,24,25,26,27,28,29,30,31] +
                  [10,11,10,12,13,14,15,16,16,17,18,18,19,20])
})

# Demo: silver_events (purchase events per user)
silver_events = pd.DataFrame({
    "user_id": [101,101,101,102,102,103,104,104,104,104,105,105,105,105,105,106,107,107],
    "event_type": ["purchase"]*18,
    "price": [500, 700, 800, 1200, 1500, 300, 200, 250, 300, 400, 100,150,200,250,300, 900, 50, 60]
})

con.register("gold_products_df", gold_products)
con.register("silver_events_df", silver_events)

con.execute("CREATE TABLE gold_products AS SELECT * FROM gold_products_df;")
con.execute("CREATE TABLE silver_events AS SELECT * FROM silver_events_df;")

print(" Demo tables created: gold_products, silver_events")

# ==========================================================
#  1) Revenue with 7-day moving average
# ==========================================================
query_ma7 = """
WITH daily AS (
  SELECT event_date, SUM(revenue) AS rev
  FROM gold_products
  GROUP BY event_date
)
SELECT
  event_date,
  rev,
  ROUND(
    AVG(rev) OVER (
      ORDER BY event_date
      ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ), 2
  ) AS ma7
FROM daily
ORDER BY event_date;
"""

ma7_df = con.execute(query_ma7).df()
print("\n 1) Revenue with 7-day moving average (MA7):")
print(ma7_df.head(10))

# ==========================================================
#  2) Conversion funnel by category
# ==========================================================
query_funnel = """
SELECT
  category_code,
  SUM(views) AS views,
  SUM(purchases) AS purchases,
  ROUND(SUM(purchases) * 100.0 / NULLIF(SUM(views), 0), 2) AS conversion_rate
FROM gold_products
GROUP BY category_code
ORDER BY conversion_rate DESC;
"""

funnel_df = con.execute(query_funnel).df()
print("\n 2) Conversion funnel (views → purchases) by category:")
print(funnel_df)

# ==========================================================
#  3) Customer tiers (VIP / Loyal / Regular)
# ==========================================================
query_tiers = """
WITH customer_summary AS (
  SELECT
    user_id,
    COUNT(*) AS cnt,
    SUM(price) AS total_spent
  FROM silver_events
  WHERE event_type = 'purchase'
  GROUP BY user_id
),
tiered AS (
  SELECT
    user_id,
    cnt,
    total_spent,
    CASE
      WHEN cnt >= 10 THEN 'VIP'
      WHEN cnt >= 5 THEN 'Loyal'
      ELSE 'Regular'
    END AS tier
  FROM customer_summary
)
SELECT
  tier,
  COUNT(*) AS customers,
  ROUND(AVG(total_spent), 2) AS avg_ltv,
  ROUND(SUM(total_spent), 2) AS total_spent
FROM tiered
GROUP BY tier
ORDER BY total_spent DESC;
"""

tiers_df = con.execute(query_tiers).df()
print("\n 3) Customer tiers summary:")
print(tiers_df)

print("\n Task 9 Completed Successfully!")


 Demo tables created: gold_products, silver_events

 1) Revenue with 7-day moving average (MA7):
  event_date     rev      ma7
0 2024-01-01  2100.0  2100.00
1 2024-01-02  2270.0  2185.00
2 2024-01-03  1830.0  2066.67
3 2024-01-04  2530.0  2182.50
4 2024-01-05  2660.0  2278.00
5 2024-01-06  2430.0  2303.33
6 2024-01-07  2800.0  2374.29
7 2024-01-08  2940.0  2494.29
8 2024-01-09  2890.0  2582.86
9 2024-01-10  2790.0  2720.00

 2) Conversion funnel (views → purchases) by category:
  category_code    views  purchases  conversion_rate
0   electronics  18070.0      746.0             4.13
1       fashion  14650.0      343.0             2.34
2       grocery  12280.0      209.0             1.70

 3) Customer tiers summary:
      tier  customers  avg_ltv  total_spent
0  Regular          6  1193.33       7160.0
1    Loyal          1  1000.00       1000.0

 Task 9 Completed Successfully!
